In [ ]:
%load_ext autoreload
%autoreload 2

In [ ]:
from nanomesh.volume import Volume
from nanomesh.plane import Plane

In [ ]:
vol = Volume.load('sample_data.npy')

In [ ]:
vol.show_slice()

In [ ]:
plane = vol.select_plane(x=99)

In [ ]:
plane.show()

In [ ]:
from skimage import filters

In [ ]:
plane_gauss = plane.apply_np(filters.gaussian, sigma=5)
plane_gauss.show()

In [ ]:
filters.try_all_threshold(plane_gauss.array_view, figsize=(10, 15), verbose=True)

In [ ]:
thresh = filters.threshold_li(plane_gauss.array_view)

seg = Plane.from_array(1.0 * (plane_gauss.array_view >= thresh))
seg.show()

In [ ]:
# https://forum.image.sc/t/create-3d-volume-mesh/34052/9

from skimage.measure import find_contours, approximate_polygon
from scipy.spatial import Delaunay
import numpy as np
import matplotlib.pyplot as plt

contours = find_contours(seg.array_view, 0)
contours = [approximate_polygon(contour, 1) for contour in contours]
vertices = np.vstack(contours)
triangles = Delaunay(vertices).simplices
tri_x, tri_y, tri_z = triangles.T
centers = (vertices[tri_x] + vertices[tri_y] + vertices[tri_z])/3
mask = seg.array_view[tuple(centers.astype(int).T)] == 0
x, y = vertices.T[::-1]

plt.figure(figsize=(12, 12))

ax = plt.subplot(221)
ax.set_title('Segmented image')
ax.imshow(seg.array_view, cmap='gray')

ax = plt.subplot(222)
ax.set_title(f'Contours ({len(contours)})')
ax.imshow(seg.array_view, cmap='gray')
for contour in contours: 
    ax.plot(*contour.T[::-1], 'red')

ax = plt.subplot(223)
ax.set_title(f'All triangles ({len(triangles)})')
ax.imshow(seg.array_view, cmap='gray')
ax.triplot(*vertices.T[::-1], triangles)

ax = plt.subplot(224)
ax.set_title(f'Filtered triangles ({mask.sum()})')
ax.imshow(seg.array_view, cmap='gray')
ax.triplot(x, y, triangles=triangles, mask=mask)
plt.show()

In [ ]:
from matplotlib.tri.triplot import Triangulation
tri, _, _ = Triangulation.get_from_args_and_kwargs(x, y, triangles=triangles, mask=mask)

In [ ]:
import numpy
import pygalmesh
import pyvista as pv

points = numpy.array([[0.0, 0.0], [1.0, 0.0], [1.0, 1.0], [0.0, 1.0]])
constraints = [[0, 1], [1, 2], [2, 3], [3, 0]]

mesh = pygalmesh.generate_2d(
    vertices,
    tri.edges,
    max_edge_size=50,
    num_lloyd_steps=10,
)
# mesh.points, mesh.cells

pv.plot_itk(mesh)

### TODO

- Generate meshio object directly from triangles / edges?
- Add box around image to avoid problems in the corners

In [ ]:
vertices?

In [ ]:
pygalmesh.generate_2d?